In [ ]:
!wget https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel

--2024-06-08 15:07:23--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55156 (54K)
Saving to: ‘integrated_call_samples_v3.20130502.ALL.panel’

integrated_call_sam 100%[===================>]  53.86K   345KB/s    in 0.2s    

2024-06-08 15:07:24 (345 KB/s) - ‘integrated_call_samples_v3.20130502.ALL.panel’ saved [55156/55156]



In [ ]:
!wget -nH --cut-dirs=4 -r -p -e robots=off -A 'ALL.*' https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/

--2024-06-08 12:48:20--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.tmp’

index.html.tmp          [ <=>                ]  19.90K  --.-KB/s    in 0.08s   

2024-06-08 12:48:20 (256 KB/s) - ‘index.html.tmp’ saved [20382]

Removing index.html.tmp since it should be rejected.

--2024-06-08 12:48:20--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/?C=N;O=D
Reusing existing connection to ftp.1000genomes.ebi.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html?C=N;O=D.tmp’

index.html?C=N;O=D.     [ <=>                ]  19.90K  --.-KB/s    in 0.04s   

2024-06-08 12:48:20 (545 KB/s) - ‘index.html?C=N;O=D.tmp’ saved [2038

Практическая работа. Работа с библиотекой.

In [ ]:
pip install hail

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing

In [ ]:
import hail as hl
from hail.plot import show
from pprint import pprint

Prepare - удаляем группу AMR (смешанные американцы) и создаем тестовую и обучающую выборки

In [ ]:
!grep -E 'AFR|EUR|SAS|EAS' annotation.txt > annotation_noAMR.txt

STEP0 - отбираем только би-аллельные SNP и убирает инделы

In [ ]:
chr1 = hl.import_vcf('ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz', force_bgz=True)

In [ ]:
#only bi-allelic SNP
chr1 = chr1.filter_rows(hl.len(chr1.alleles) == 2)
#remove indels
chr1 = chr1.filter_rows(hl.is_snp(chr1.alleles[0], chr1.alleles[1]))

STEP1 - считаем частоты SNP

In [ ]:
table = hl.import_table("integrated_call_samples_v3.20130502.ALL.panel").key_by("sample")
mt = chr1

# объединяем данные из аннотации с данными из vcf файла
mt = mt.annotate_cols(pheno=table[mt.s])

#Список популяций в файле
pops = list(table.aggregate(hl.agg.collect_as_set(table.super_pop)))

# Инициализация таблицы первой популяцией
mt_pop = mt.filter_cols(mt.pheno.super_pop == pops[0])
mt_pop = hl.variant_qc(mt_pop)
mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AF[1])
freq_table = mt_pop.rows().select('alt').rename({"alt": f'{pops[0]}'})

# Цикл по оставшимся популяциям
for pop in pops[1:]:
  mt_pop = mt.filter_cols(mt.pheno.super_pop == pop)
  mt_pop = hl.variant_qc(mt_pop)
  mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AF[1])
  pop_freq = mt_pop.rows().select('alt').rename({"alt": f'{pop}'})

  # Объединяем таблицы
  freq_table = freq_table.join(pop_freq)

STEP2 - фильтрация SNP по информативности

In [ ]:
#Задаем уровень информативности, при котором мы оставляем SNP
informativenessBound = 0.9
#Вводим дополнительные функции для подсчета информативности
def xlogx2(x):
    z = xlogx(x) + xlogx(1 - x)
    return z

def xlogx(x):
    z = hl.if_else(x == 0, 0, x * hl.log(x))
    return z
#Считаем количесвто строк в файле
nss = freq_table.count()
#Считаем средние
freq_table = freq_table.annotate(
    mean_row=(freq_table.AFR + freq_table.SAS + freq_table.EUR + freq_table.AMR + freq_table.EAS) / len(pops)
)
freq_table = freq_table.annotate(
    mean_row_log=(xlogx2(freq_table.AFR) + xlogx2(freq_table.SAS) + xlogx2(freq_table.EUR) + xlogx2(freq_table.AMR) + xlogx2(freq_table.EAS)) / len(pops)
)
#Считаем информативность
freq_table = freq_table.annotate(info= -xlogx2(freq_table.mean_row) + freq_table.mean_row_log)
#Фильтруем таблицу по информативности (в порядке убывания) и отбираем 10% самых информативных SNP
informativenessQuantile = freq_table.order_by(hl.desc(freq_table.info)).head(hl.eval(hl.int32(hl.floor((1 - informativenessBound)* nss))))
#Создаем новый vcf-файл с отобранными SNP по информативности
informativenessQuantile = informativenessQuantile.key_by("locus")
mt = chr1.filter_rows(hl.is_defined(informativenessQuantile[chr1.locus]))
#Сохраняем файл
hl.export_vcf(mt, 'tmp/filter_chr1.vcf.bgz')

STEP3 - делим имеющийся файл на маленькие

In [ ]:
mt = hl.import_vcf('tmp/filter_chr1.vcf.bgz')
stepsize = 10000
# Вычисляем количество сегментов и преобразуем в список
segments = hl.range(0, hl.int32((mt.count_rows() - 1) // stepsize + 1)).collect()
# Добавляем столбец с номером строки
mt = mt.annotate_rows(row_index=hl.scan.count())

# Фильтруем строки по номеру

# Цикл по сегментам
for j in segments[0]:
    start = j * stepsize
    end = hl.eval(min((j + 1) * stepsize, mt.count_rows()))
    # Фильтруем строки по номеру
    segment_mt = mt.filter_rows((mt.row.row_index >= start) & (mt.row.row_index < end))
    hl.export_vcf(segment_mt, f"tmp/chr_segment{j}.vcf.bgz")

STEP4 - считаем кол-во строк в новом vcf файле

In [ ]:
nss = hl.eval(chr.count()[0])